In [5]:
from gurobipy import Model, GRB, quicksum

# Data
locations = range(1, 101)  # 100 locations
delivery_options = [1, 2, 3, 5]  
WW = {i: 100 + i for i in locations} 
C_15 = 150
C_10 = 120
truck_capacities = {15: 15, 10: 10}

# Create a Gurobi model
model = Model("Truck_Delivery_Optimization")

# Decision variables
x_15 = model.addVars(locations, delivery_options, vtype=GRB.CONTINUOUS, name="x_15")
x_10 = model.addVars(locations, delivery_options, vtype=GRB.CONTINUOUS, name="x_10")

# Objective function: Minimize total delivery cost
model.setObjective(
    quicksum((x_15[i, d] / truck_capacities[15]) * C_15 + (x_10[i, d] / truck_capacities[10]) * C_10
             for i in locations for d in delivery_options),
    GRB.MINIMIZE
)

# Constraints
# 1. Weight Delivery Constraint
for i in locations:
    model.addConstr(
        quicksum(x_15[i, d] + x_10[i, d] for d in delivery_options) == WW[i],
        name=f"Weight_Delivery_Constraint_{i}"
    )

# 2. Truck Capacity Constraints
for i in locations:
    for d in delivery_options:
        model.addConstr(x_15[i, d] <= truck_capacities[15] * d, name=f"Capacity_15_{i}_{d}")
        model.addConstr(x_10[i, d] <= truck_capacities[10] * d, name=f"Capacity_10_{i}_{d}")

# 3. Non-Negativity is automatically handled by Gurobi since the variables are continuous and non-negative

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for i in locations:
        for d in delivery_options:
            print(f"Location {i}, Delivery {d} times a week:")
            print(f"  15-ton truck delivers: {x_15[i, d].X} tons")
            print(f"  10-ton truck delivers: {x_10[i, d].X} tons")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 900 rows, 800 columns and 1600 nonzeros
Model fingerprint: 0xa5992f48
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 900 rows and 800 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5176000e+05   0.000000e+00   4.000000e+00      0s
Extra simplex iterations after uncrush: 2
       2    1.5176000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.517600000e+05
Location 1, Delivery 1 times a week:
  15-ton truck delivers: 0.0 tons
  10

In [8]:
import gurobipy as gp
from gurobipy import GRB
import random

# Sample data (you would replace this with your actual data)
num_locations = 100
W = [random.randint(50, 200) for _ in range(num_locations)]  # Total weekly weight for each location
C15 = 150  # Cost of using 15-ton truck per trip
C10 = 120  # Cost of using 10-ton truck per trip
days = 5  # Number of days (Monday to Friday)
delivery_options = {'once': 1, 'twice': 2, 'thrice': 3, 'five_times': 5}  # Delivery options

# Initialize model
model = gp.Model("Truck_Delivery_Optimization")

# Decision variables
x = model.addVars(num_locations, len(delivery_options), vtype=GRB.BINARY, name="x")
y15 = model.addVars(num_locations, days, vtype=GRB.CONTINUOUS, name="y15")
y10 = model.addVars(num_locations, days, vtype=GRB.CONTINUOUS, name="y10")

# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum((y15[i, j] / 15) * C15 + (y10[i, j] / 10) * C10 for i in range(num_locations) for j in range(days)),
    GRB.MINIMIZE
)

# Constraints

# 1. Delivery frequency constraint
model.addConstrs(
    gp.quicksum(x[i, k] for k in range(len(delivery_options))) == 1 for i in range(num_locations)
)

# 2. Total weekly delivered weight constraint
model.addConstrs(
    gp.quicksum(y15[i, j] + y10[i, j] for j in range(days)) == W[i] for i in range(num_locations)
)

# 3. Daily delivery constraints
for i in range(num_locations):
    for j in range(days):
        model.addConstr(y15[i, j] + y10[i, j] <= W[i])

# 4. Truck capacity constraints
model.addConstrs(y15[i, j] <= 15 for i in range(num_locations) for j in range(days))
model.addConstrs(y10[i, j] <= 10 for i in range(num_locations) for j in range(days))

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal total cost: {model.objVal}")
    for i in range(num_locations):
        print(f"Location {i+1}:")
        for j in range(days):
            print(f"  Day {j+1} - 15-ton: {y15[i,j].x}, 10-ton: {y10[i,j].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1700 rows, 1400 columns and 3400 nonzeros
Model fingerprint: 0x92093cb4
Variable types: 1000 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1600 rows and 400 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.
